# Week 4: Sensor Technologies & Fusion

## Module II: Perception & Localization

### Topics Covered

- Camera (Image Processing, Homography)
- LiDAR (Point Clouds, Range Data)
- Radar (Doppler Effects)
- Sensor Synchronization and Calibration

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. Understand the physics and characteristics of automotive sensors (Camera, LiDAR, Radar)
2. Process camera images and apply homography transformations
3. Work with LiDAR point clouds and range data
4. Understand radar principles and Doppler effect
5. Perform sensor calibration and extrinsic/intrinsic parameter estimation
6. Implement multi-sensor fusion techniques
7. Synchronize data from multiple sensors with different frequencies

---

## Setup

Import required libraries for sensor data processing and visualization

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, Rectangle, Wedge
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial.transform import Rotation

# Set random seed
np.random.seed(42)

# Plotting configuration
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10

print("Libraries loaded successfully!")
print("NumPy version:", np.__version__)

## 1. Camera Sensors

**Cameras** are the most cost-effective sensors for autonomous vehicles, providing rich visual information about the environment including colors, textures, lane markings, traffic signs, and lights.

### Camera Basics

#### **Pinhole Camera Model**

The fundamental model relating 3D world points to 2D image points:

$$\begin{bmatrix} u \\ v \\ 1 \end{bmatrix} = \frac{1}{Z} \mathbf{K} \begin{bmatrix} X \\ Y \\ Z \end{bmatrix}$$

Where:
- **(X, Y, Z)**: 3D point in camera frame
- **(u, v)**: 2D pixel coordinates
- **K**: Intrinsic camera matrix
- **Z**: Depth (distance from camera)

#### **Intrinsic Parameters (K Matrix)**

$$\mathbf{K} = \begin{bmatrix} f_x & 0 & c_x \\ 0 & f_y & c_y \\ 0 & 0 & 1 \end{bmatrix}$$

- **fx, fy**: Focal lengths (in pixels)
- **cx, cy**: Principal point (optical center)

**Typical values** for automotive cameras:
- Focal length: 700-1200 pixels
- Resolution: 1920×1080 (Full HD) or 1280×720 (HD)
- Field of View (FoV): 50-120 degrees
- Frame rate: 30-60 FPS

---

### Camera Types in Autonomous Vehicles

| Camera Type | FoV | Use Case | Position |
|-------------|-----|----------|----------|
| **Front Wide** | 120° | Lane keeping, traffic lights | Windshield |
| **Front Narrow** | 50° | Long-range detection (200m+) | Windshield |
| **Side** | 90° | Lane changes, blind spots | Side mirrors |
| **Rear** | 120° | Parking, rear monitoring | Rear bumper |
| **Fisheye** | 180°+ | 360° surround view | Roof/corners |

---

### Image Processing Pipeline

```
Raw Image → Undistortion → Color Space → Feature Extraction → Object Detection
                           Conversion
```

#### **1. Lens Distortion Correction**

Real lenses have radial and tangential distortion. Undistortion formula:

$$\begin{align}
x_{distorted} &= x(1 + k_1 r^2 + k_2 r^4 + k_3 r^6) \\
y_{distorted} &= y(1 + k_1 r^2 + k_2 r^4 + k_3 r^6)
\end{align}$$

Where **r² = x² + y²** and **k₁, k₂, k₃** are radial distortion coefficients.

#### **2. Color Spaces**
- **RGB**: Raw camera output
- **HSV**: Better for color-based segmentation (e.g., lane lines)
- **Grayscale**: Reduces data, used for edge detection

#### **3. Edge Detection**
- **Canny Edge Detector**: Multi-stage algorithm for edge detection
- **Sobel Filter**: Gradient-based edge detection

---

### Homography: Ground Plane Projection

**Homography** transforms a planar surface from one view to another. Critical for **bird's-eye view** (BEV) transformation.

#### **Homography Matrix (3×3)**

$$\mathbf{H} = \begin{bmatrix} h_{11} & h_{12} & h_{13} \\ h_{21} & h_{22} & h_{23} \\ h_{31} & h_{32} & h_{33} \end{bmatrix}$$

Maps image point **(u, v)** to ground plane point **(x, y)**:

$$\begin{bmatrix} x \\ y \\ 1 \end{bmatrix} \sim \mathbf{H} \begin{bmatrix} u \\ v \\ 1 \end{bmatrix}$$

#### **Computing Homography**

Given 4+ corresponding points between image and ground plane, solve for H using:
- **Direct Linear Transform (DLT)**
- **RANSAC** for robustness to outliers

**Application**: Convert front camera view → top-down view for lane detection

---

### Advantages & Limitations

**Advantages**:
- ✅ Low cost ($50-200 per camera)
- ✅ Rich semantic information (colors, textures, text)
- ✅ High resolution (megapixels)
- ✅ Passive sensor (no emissions)

**Limitations**:
- ❌ No direct depth information (monocular)
- ❌ Performance degrades in poor lighting, rain, fog
- ❌ Sensitive to sun glare
- ❌ Requires heavy computation for deep learning

**Solutions**:
- Stereo cameras for depth
- Multi-sensor fusion (camera + LiDAR)
- HDR imaging for varying light conditions

In [ ]:
# Camera Homography Demonstration

class CameraModel:
    """Simple pinhole camera model."""
    
    def __init__(self, fx=800, fy=800, cx=640, cy=360, width=1280, height=720):
        """
        Args:
            fx, fy: Focal lengths in pixels
            cx, cy: Principal point (optical center)
            width, height: Image resolution
        """
        self.K = np.array([
            [fx, 0, cx],
            [0, fy, cy],
            [0, 0, 1]
        ])
        self.width = width
        self.height = height
        
    def project_3d_to_2d(self, points_3d):
        """
        Project 3D points (in camera frame) to 2D image coordinates.
        
        Args:
            points_3d: Nx3 array of 3D points [X, Y, Z]
        
        Returns:
            points_2d: Nx2 array of pixel coordinates [u, v]
        """
        # Homogeneous coordinates
        points_2d_hom = (self.K @ points_3d.T).T
        
        # Normalize by Z (depth)
        points_2d = points_2d_hom[:, :2] / points_2d_hom[:, 2:3]
        
        return points_2d


# Simulate camera viewing a road scene
camera = CameraModel()

# Define 3D points on ground plane (Z=0, camera at height 1.5m)
# Road lanes in front of vehicle
camera_height = 1.5  # meters above ground

# Create grid of points on ground plane
x_range = np.linspace(0, 50, 20)  # 0-50m in front
y_range = np.linspace(-10, 10, 10)  # ±10m width

ground_points = []
for x in x_range:
    for y in y_range:
        # Points in camera frame: X=forward, Y=left, Z=up
        # Ground plane: Z = -camera_height (below camera)
        ground_points.append([x, y, -camera_height])

ground_points = np.array(ground_points)

# Only keep points with positive depth (in front of camera)
valid_mask = ground_points[:, 0] > 0
ground_points_valid = ground_points[valid_mask]

# Project to image
image_points = camera.project_3d_to_2d(ground_points_valid)

# Filter points within image bounds
in_image = (image_points[:, 0] >= 0) & (image_points[:, 0] < camera.width) & \
           (image_points[:, 1] >= 0) & (image_points[:, 1] < camera.height)

image_points_valid = image_points[in_image]
ground_points_final = ground_points_valid[in_image]

# Visualization
fig = plt.figure(figsize=(16, 7))

# Left: Front camera view
ax1 = fig.add_subplot(1, 2, 1)
ax1.scatter(image_points_valid[:, 0], image_points_valid[:, 1], c=ground_points_final[:, 0], 
           cmap='viridis', s=50, alpha=0.6)
ax1.set_xlim([0, camera.width])
ax1.set_ylim([camera.height, 0])  # Flip Y for image coordinates
ax1.set_xlabel('u (pixels)', fontweight='bold')
ax1.set_ylabel('v (pixels)', fontweight='bold')
ax1.set_title('Front Camera View (Perspective)', fontweight='bold', fontsize=14)
ax1.set_aspect('equal')
ax1.grid(True, alpha=0.3)

# Add lane markings
lane_left_3d = np.array([[i, 3.5, -camera_height] for i in range(5, 50, 2)])
lane_right_3d = np.array([[i, -3.5, -camera_height] for i in range(5, 50, 2)])

lane_left_2d = camera.project_3d_to_2d(lane_left_3d)
lane_right_2d = camera.project_3d_to_2d(lane_right_3d)

ax1.plot(lane_left_2d[:, 0], lane_left_2d[:, 1], 'y-', linewidth=3, label='Left Lane')
ax1.plot(lane_right_2d[:, 0], lane_right_2d[:, 1], 'y-', linewidth=3, label='Right Lane')
ax1.legend()

# Right: Bird's Eye View (top-down)
ax2 = fig.add_subplot(1, 2, 2)
ax2.scatter(ground_points_final[:, 0], ground_points_final[:, 1], 
           c=ground_points_final[:, 0], cmap='viridis', s=50, alpha=0.6)
ax2.set_xlabel('X (m) - Forward', fontweight='bold')
ax2.set_ylabel('Y (m) - Left', fontweight='bold')
ax2.set_title('Bird\'s Eye View (Top-Down)', fontweight='bold', fontsize=14)
ax2.set_aspect('equal')
ax2.grid(True, alpha=0.3)

# Add lane markings
ax2.plot(lane_left_3d[:, 0], lane_left_3d[:, 1], 'y-', linewidth=3, label='Left Lane')
ax2.plot(lane_right_3d[:, 0], lane_right_3d[:, 1], 'y-', linewidth=3, label='Right Lane')

# Add vehicle position
ax2.plot(0, 0, 'ro', markersize=15, label='Vehicle')
vehicle_rect = Rectangle((-2, -1), 4, 2, fill=False, edgecolor='red', linewidth=2)
ax2.add_patch(vehicle_rect)

ax2.legend()
ax2.set_xlim([-5, 50])
ax2.set_ylim([-15, 15])

plt.tight_layout()
plt.show()

print("=" * 70)
print("Camera Parameters:")
print("=" * 70)
print(f"Intrinsic Matrix K:\n{camera.K}\n")
print(f"Focal lengths: fx={camera.K[0,0]:.1f} px, fy={camera.K[1,1]:.1f} px")
print(f"Principal point: cx={camera.K[0,2]:.1f} px, cy={camera.K[1,2]:.1f} px")
print(f"Resolution: {camera.width}×{camera.height}")
print(f"Camera height: {camera_height} m")
print("=" * 70)

## 2. LiDAR Sensors

**LiDAR (Light Detection and Ranging)** uses laser pulses to measure distances and create precise 3D maps of the environment. It's the cornerstone of most autonomous vehicle sensor suites.

### LiDAR Basics

#### **Operating Principle**

1. **Emit**: Laser pulse sent out
2. **Reflect**: Pulse bounces off object
3. **Receive**: Sensor detects reflected pulse
4. **Calculate**: Distance = (Time of Flight × Speed of Light) / 2

$$d = \frac{c \cdot \Delta t}{2}$$

Where:
- **d**: Distance to object
- **c**: Speed of light (≈ 3 × 10⁸ m/s)
- **Δt**: Time of flight (round trip)

**Example**: For Δt = 200 nanoseconds:
- d = (3 × 10⁸ m/s × 200 × 10⁻⁹ s) / 2 = 30 meters

---

### LiDAR Types

| Type | Mechanism | Range | Points/sec | Cost | Use Case |
|------|-----------|-------|------------|------|----------|
| **Mechanical** | Rotating mirror/head | 100-200m | 1-2M | $$$$$ | Long-range, 360° |
| **Solid-State** | MEMS mirrors, no rotation | 50-100m | 100k-500k | $$$ | Compact, reliable |
| **Flash** | Illuminates entire scene | 30-50m | 10k-100k | $$ | Short-range |

**Common Automotive LiDAR**:
- **Velodyne HDL-64E**: 64 channels, 360°, 2.2M points/sec, ~200m range
- **Ouster OS1**: 64/128 channels, 360°, 1.3M points/sec
- **Luminar Iris**: 1550nm wavelength, 250m+ range

---

### Point Cloud Representation

**Point Cloud**: Collection of 3D points, each with (x, y, z, intensity)

```
Point = [x, y, z, intensity, timestamp, laser_id]
```

- **x, y, z**: 3D coordinates in LiDAR frame
- **intensity**: Reflectivity (0-255)
- **timestamp**: When point was captured
- **laser_id**: Which laser channel (for multi-channel LiDAR)

**Typical point cloud**:
- **Size**: 100,000 - 2,000,000 points per scan
- **Frequency**: 10-20 Hz
- **Data rate**: ~10-40 MB/s

---

### Coordinate Systems

**LiDAR Frame** (standard convention):
- **X**: Forward
- **Y**: Left
- **Z**: Up

**Spherical Coordinates** → Cartesian:

$$\begin{align}
x &= r \cos(\theta) \cos(\phi) \\
y &= r \cos(\theta) \sin(\phi) \\
z &= r \sin(\theta)
\end{align}$$

Where:
- **r**: Range (distance)
- **θ**: Elevation angle (vertical)
- **φ**: Azimuth angle (horizontal)

---

### Point Cloud Processing Pipeline

```
Raw Points → Ground Removal → Clustering → Object Detection → Tracking
```

#### **1. Ground Plane Removal**
- **RANSAC**: Fit plane to ground points
- **Height threshold**: Remove points below z < -1.5m

#### **2. Clustering**
- **Euclidean clustering**: Group nearby points
- **DBSCAN**: Density-based clustering

#### **3. Bounding Box Fitting**
- **Oriented bounding box (OBB)**: Fit box to cluster
- **Extract**: Position, size, orientation

---

### Advantages & Limitations

**Advantages**:
- ✅ Accurate 3D measurements (±2cm precision)
- ✅ Direct range/depth information
- ✅ Works in darkness (active sensor)
- ✅ 360° field of view (mechanical)
- ✅ Not affected by texture/color

**Limitations**:
- ❌ Expensive ($1,000 - $75,000)
- ❌ Performance degrades in rain, fog, snow
- ❌ No color information
- ❌ Lower resolution than cameras
- ❌ Moving parts can fail (mechanical)
- ❌ Difficulty detecting dark/absorbing surfaces

**Solutions**:
- Multi-wavelength LiDAR (905nm + 1550nm)
- Sensor fusion with cameras
- Solid-state for reliability

In [ ]:
# LiDAR Sensor Model and Point Cloud Processing

class LiDARSensor:
    """
    Simulates a 3D LiDAR sensor with configurable resolution and range.
    """
    def __init__(self, 
                 horizontal_resolution=0.2,  # degrees
                 vertical_resolution=2.0,    # degrees
                 horizontal_fov=360,         # degrees
                 vertical_fov=(-25, 15),     # (min, max) in degrees
                 max_range=100,              # meters
                 min_range=1):               # meters
        self.h_res = np.deg2rad(horizontal_resolution)
        self.v_res = np.deg2rad(vertical_resolution)
        self.h_fov = np.deg2rad(horizontal_fov)
        self.v_fov = (np.deg2rad(vertical_fov[0]), np.deg2rad(vertical_fov[1]))
        self.max_range = max_range
        self.min_range = min_range
        
        # Generate scan pattern
        self.azimuth_angles = np.arange(0, self.h_fov, self.h_res)
        self.elevation_angles = np.arange(self.v_fov[0], self.v_fov[1], self.v_res)
        
    def spherical_to_cartesian(self, ranges, azimuth, elevation):
        """
        Convert spherical coordinates (r, θ, φ) to Cartesian (x, y, z).
        
        Args:
            ranges: Distance measurements (m)
            azimuth: Horizontal angle φ (radians)
            elevation: Vertical angle θ (radians)
        
        Returns:
            Point cloud in Cartesian coordinates (N x 3)
        """
        x = ranges * np.cos(elevation) * np.cos(azimuth)
        y = ranges * np.cos(elevation) * np.sin(azimuth)
        z = ranges * np.sin(elevation)
        return np.column_stack([x, y, z])
    
    def scan_environment(self, objects):
        """
        Simulate LiDAR scan of environment with objects.
        
        Args:
            objects: List of (center, size, shape_type) tuples
        
        Returns:
            Point cloud (N x 4): [x, y, z, intensity]
        """
        points = []
        intensities = []
        
        for elevation in self.elevation_angles:
            for azimuth in self.azimuth_angles:
                # Ray direction
                ray_dir = np.array([
                    np.cos(elevation) * np.cos(azimuth),
                    np.cos(elevation) * np.sin(azimuth),
                    np.sin(elevation)
                ])
                
                # Find intersection with objects or ground
                min_range = self.max_range
                hit_intensity = 0.3  # Default ground intensity
                
                # Check ground plane (z = 0)
                if ray_dir[2] < 0:
                    t = -0.0 / ray_dir[2]  # sensor at z=0
                    if self.min_range < t < min_range:
                        min_range = t
                        hit_intensity = 0.4
                
                # Check objects
                for obj_center, obj_size, shape_type in objects:
                    if shape_type == 'box':
                        # Simplified box intersection
                        t = np.linalg.norm(obj_center[:2])  # Distance to center
                        if abs(np.arctan2(obj_center[1], obj_center[0]) - azimuth) < 0.1:
                            if self.min_range < t < min_range:
                                min_range = t
                                hit_intensity = 0.8
                    elif shape_type == 'sphere':
                        # Sphere intersection
                        oc = -obj_center
                        b = np.dot(oc, ray_dir)
                        c = np.dot(oc, oc) - obj_size**2
                        discriminant = b**2 - c
                        if discriminant > 0:
                            t = b - np.sqrt(discriminant)
                            if self.min_range < t < min_range:
                                min_range = t
                                hit_intensity = 0.9
                
                if min_range < self.max_range:
                    point = min_range * ray_dir
                    points.append(point)
                    intensities.append(hit_intensity)
        
        points = np.array(points)
        intensities = np.array(intensities).reshape(-1, 1)
        return np.hstack([points, intensities])
    
    def remove_ground_plane(self, point_cloud, ground_threshold=-0.2):
        """
        Remove ground points using height threshold.
        
        Args:
            point_cloud: (N x 4) array [x, y, z, intensity]
            ground_threshold: Maximum z-value for ground points
        
        Returns:
            Point cloud with ground removed
        """
        return point_cloud[point_cloud[:, 2] > ground_threshold]
    
    def cluster_points(self, point_cloud, distance_threshold=0.5):
        """
        Simple distance-based clustering of points.
        
        Args:
            point_cloud: (N x 4) array [x, y, z, intensity]
            distance_threshold: Maximum distance for same cluster
        
        Returns:
            Cluster labels (N,)
        """
        from sklearn.cluster import DBSCAN
        clustering = DBSCAN(eps=distance_threshold, min_samples=5)
        labels = clustering.fit_predict(point_cloud[:, :3])
        return labels

# Create LiDAR sensor
lidar = LiDARSensor(
    horizontal_resolution=0.4,
    vertical_resolution=2.0,
    horizontal_fov=360,
    max_range=50
)

# Define environment: (center, size, shape_type)
objects = [
    (np.array([15, 0, 0.75]), 2.0, 'box'),      # Vehicle ahead
    (np.array([20, -3, 0.75]), 2.0, 'box'),     # Vehicle left lane
    (np.array([8, 2, 0.9]), 0.3, 'sphere'),     # Pedestrian right
    (np.array([25, 5, 0.5]), 0.3, 'sphere'),    # Pedestrian far right
]

# Perform LiDAR scan
point_cloud = lidar.scan_environment(objects)
print(f"Total points in scan: {len(point_cloud)}")

# Remove ground plane
non_ground_points = lidar.remove_ground_plane(point_cloud, ground_threshold=-0.1)
print(f"Non-ground points: {len(non_ground_points)}")

# Cluster objects
if len(non_ground_points) > 0:
    cluster_labels = lidar.cluster_points(non_ground_points, distance_threshold=1.5)
    num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)
    print(f"Detected clusters: {num_clusters}")

# Visualization
fig = plt.figure(figsize=(16, 6))

# 1. Full point cloud (top view)
ax1 = fig.add_subplot(131)
scatter1 = ax1.scatter(point_cloud[:, 0], point_cloud[:, 1], 
                       c=point_cloud[:, 3], cmap='viridis', s=1, alpha=0.6)
ax1.scatter(0, 0, c='red', s=200, marker='^', label='LiDAR sensor', edgecolors='black', linewidths=2)
ax1.set_xlabel('X (m)', fontsize=11)
ax1.set_ylabel('Y (m)', fontsize=11)
ax1.set_title('LiDAR Point Cloud - Top View', fontsize=13, fontweight='bold')
ax1.set_aspect('equal')
ax1.grid(True, alpha=0.3)
ax1.legend()
cbar1 = plt.colorbar(scatter1, ax=ax1)
cbar1.set_label('Intensity', fontsize=10)
ax1.set_xlim(-10, 40)
ax1.set_ylim(-20, 20)

# 2. 3D point cloud
ax2 = fig.add_subplot(132, projection='3d')
scatter2 = ax2.scatter(point_cloud[:, 0], point_cloud[:, 1], point_cloud[:, 2],
                       c=point_cloud[:, 2], cmap='terrain', s=1, alpha=0.5)
ax2.scatter(0, 0, 0, c='red', s=200, marker='^', label='LiDAR sensor')
ax2.set_xlabel('X (m)', fontsize=10)
ax2.set_ylabel('Y (m)', fontsize=10)
ax2.set_zlabel('Z (m)', fontsize=10)
ax2.set_title('LiDAR Point Cloud - 3D View', fontsize=13, fontweight='bold')
ax2.legend()
cbar2 = plt.colorbar(scatter2, ax=ax2, pad=0.1, shrink=0.8)
cbar2.set_label('Height (m)', fontsize=10)
ax2.view_init(elev=25, azim=45)

# 3. Clustered objects (ground removed)
ax3 = fig.add_subplot(133)
if len(non_ground_points) > 0:
    scatter3 = ax3.scatter(non_ground_points[:, 0], non_ground_points[:, 1],
                          c=cluster_labels, cmap='tab10', s=3, alpha=0.7)
    ax3.scatter(0, 0, c='red', s=200, marker='^', label='LiDAR sensor', edgecolors='black', linewidths=2)
    
    # Draw bounding boxes for each cluster
    for cluster_id in set(cluster_labels):
        if cluster_id == -1:  # Skip noise
            continue
        cluster_points = non_ground_points[cluster_labels == cluster_id]
        x_min, x_max = cluster_points[:, 0].min(), cluster_points[:, 0].max()
        y_min, y_max = cluster_points[:, 1].min(), cluster_points[:, 1].max()
        rect = plt.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min,
                            fill=False, edgecolor='red', linewidth=2)
        ax3.add_patch(rect)
    
ax3.set_xlabel('X (m)', fontsize=11)
ax3.set_ylabel('Y (m)', fontsize=11)
ax3.set_title('Object Detection (Ground Removed)', fontsize=13, fontweight='bold')
ax3.set_aspect('equal')
ax3.grid(True, alpha=0.3)
ax3.legend()
ax3.set_xlim(-5, 35)
ax3.set_ylim(-10, 10)

plt.tight_layout()
plt.show()

# Print cluster statistics
print("\n=== Detected Objects ===")
for cluster_id in set(cluster_labels):
    if cluster_id == -1:
        continue
    cluster_points = non_ground_points[cluster_labels == cluster_id]
    center = cluster_points[:, :3].mean(axis=0)
    size = np.max(np.ptp(cluster_points[:, :3], axis=0))
    distance = np.linalg.norm(center[:2])
    print(f"Cluster {cluster_id}: Center=({center[0]:.1f}, {center[1]:.1f}, {center[2]:.1f})m, "
          f"Size={size:.1f}m, Distance={distance:.1f}m, Points={len(cluster_points)}")

## 3. Radar Sensors

**Radar (Radio Detection and Ranging)** uses radio waves to detect objects and measure their velocity. It's essential for adaptive cruise control (ACC) and collision avoidance systems.

### Radar Basics

#### **Operating Principle**

1. **Transmit**: Radio wave (typically 24 GHz, 77 GHz, or 79 GHz)
2. **Reflect**: Wave bounces off target
3. **Receive**: Reflected wave with frequency shift
4. **Measure**: Range and velocity simultaneously

**Key advantage**: Direct velocity measurement via Doppler effect

---

### Frequency Bands for Automotive Radar

| Band | Frequency | Wavelength | Range | Use Case |
|------|-----------|------------|-------|----------|
| **24 GHz** | 24.05-24.25 GHz | 12.5 mm | Short (0.2-30m) | Blind spot, parking |
| **77 GHz** | 76-77 GHz | 3.9 mm | Long (1-250m) | ACC, AEB, highway |
| **79 GHz** | 77-81 GHz | 3.8 mm | Medium-long | High resolution |

**Trend**: Shifting from 24 GHz → 77/79 GHz for better resolution and longer range

---

### Doppler Effect

The **Doppler shift** is the change in frequency due to relative motion between radar and target.

#### **Doppler Frequency Shift**

$$f_d = \frac{2 v_r f_0}{c}$$

Where:
- **f_d**: Doppler shift (Hz)
- **v_r**: Radial velocity (m/s) - positive = approaching, negative = receding
- **f_0**: Transmitted frequency (Hz)
- **c**: Speed of light (3 × 10⁸ m/s)

**Example** (77 GHz radar):
- Target approaching at 30 m/s (108 km/h)
- f_d = (2 × 30 × 77×10⁹) / (3×10⁸) = **15.4 kHz**

**Key insight**: Doppler shift is proportional to velocity, so radar directly measures speed!

---

### FMCW Radar (Frequency-Modulated Continuous Wave)

Most automotive radars use **FMCW** to measure both range and velocity.

#### **Principle**

1. **Transmit**: Frequency sweeps linearly (chirp)
   - Start frequency: f₀
   - Bandwidth: B (e.g., 4 GHz)
   - Chirp duration: T (e.g., 100 μs)

2. **Receive**: Delayed and Doppler-shifted signal

3. **Mix**: Transmitted and received signals → beat frequency

4. **FFT**: Extract range and velocity from beat frequency

#### **Range Calculation**

$$R = \frac{c \cdot f_b \cdot T}{2B}$$

Where:
- **f_b**: Beat frequency (Hz)
- **B**: Bandwidth (Hz)
- **T**: Chirp duration (s)

**Resolution**:
- **Range resolution**: Δr = c / (2B)
  - For B = 4 GHz: Δr = 3.75 cm
- **Velocity resolution**: Δv = λ / (2 T_frame)
  - For 77 GHz, T_frame = 50 ms: Δv ≈ 0.1 m/s

---

### Radar Characteristics

| Parameter | Typical Value | Notes |
|-----------|---------------|-------|
| **Range** | 0.2 - 250 m | Long-range: 250m, Short-range: 30m |
| **Accuracy** | ±0.1 m (range), ±0.1 m/s (velocity) | Very accurate |
| **FoV (Azimuth)** | ±60° (short), ±15° (long) | Narrow beam for long-range |
| **FoV (Elevation)** | ±10° | Limited vertical coverage |
| **Update rate** | 10-50 Hz | Fast updates |
| **Angular resolution** | 1-15° | Coarse compared to LiDAR |

---

### Radar Processing Pipeline

```
Raw IF Signal → FFT (Range) → FFT (Doppler) → CFAR Detection → Clustering → Tracking
```

#### **1. Range-Doppler Map**
- 2D FFT of FMCW signal
- Rows: Range bins
- Columns: Doppler (velocity) bins

#### **2. CFAR (Constant False Alarm Rate)**
- Adaptive thresholding to detect targets
- Distinguishes real targets from noise/clutter

#### **3. Angle Estimation**
- Multiple antennas (MIMO array)
- Digital beamforming to estimate angle of arrival (AoA)

---

### Advantages & Limitations

**Advantages**:
- ✅ Direct velocity measurement (Doppler)
- ✅ Long range (250m+)
- ✅ Works in all weather (rain, fog, snow)
- ✅ Works in darkness
- ✅ Penetrates dust, smoke
- ✅ Low cost ($50-200)
- ✅ Small size, low power

**Limitations**:
- ❌ Low angular resolution (1-15°)
- ❌ No height information (2D)
- ❌ Difficulty detecting stationary objects (ground clutter)
- ❌ Multipath reflections (metallic environments)
- ❌ Interference from other radars

**Solutions**:
- High-resolution MIMO radar (4D imaging radar)
- Sensor fusion with camera/LiDAR for angular precision
- Advanced clutter suppression algorithms

In [ ]:
# FMCW Radar Simulation with Doppler Effect

class FMCWRadar:
    """
    Simulates an FMCW (Frequency-Modulated Continuous Wave) radar.
    """
    def __init__(self, 
                 frequency=77e9,        # Hz (77 GHz)
                 bandwidth=4e9,         # Hz (4 GHz)
                 chirp_duration=100e-6, # seconds (100 μs)
                 max_range=250,         # meters
                 angular_fov=30,        # degrees (±15°)
                 angular_resolution=2): # degrees
        self.f0 = frequency
        self.B = bandwidth
        self.T = chirp_duration
        self.max_range = max_range
        self.c = 3e8  # Speed of light (m/s)
        self.wavelength = self.c / self.f0
        
        # Angular parameters
        self.angular_fov = np.deg2rad(angular_fov)
        self.angular_res = np.deg2rad(angular_resolution)
        
        # Range and velocity resolution
        self.range_resolution = self.c / (2 * self.B)
        self.velocity_resolution = self.wavelength / (2 * self.T * 10)  # Assuming 10 chirps per frame
        
    def calculate_beat_frequency(self, range_m):
        """Calculate beat frequency for a given range."""
        return (2 * self.B * range_m) / (self.c * self.T)
    
    def calculate_doppler_shift(self, velocity_mps):
        """
        Calculate Doppler frequency shift for a given radial velocity.
        Positive velocity = approaching, negative = receding.
        """
        return (2 * velocity_mps * self.f0) / self.c
    
    def detect_targets(self, objects, ego_velocity=0):
        """
        Detect targets and compute their range, velocity, and angle.
        
        Args:
            objects: List of (position, velocity, rcs) tuples
                - position: [x, y] in radar frame
                - velocity: [vx, vy] velocity vector
                - rcs: Radar cross-section (m²)
            ego_velocity: Ego vehicle velocity (m/s, forward positive)
        
        Returns:
            detections: List of (range, radial_velocity, angle, rcs) tuples
        """
        detections = []
        
        for pos, vel, rcs in objects:
            # Calculate range
            range_m = np.linalg.norm(pos)
            
            if range_m > self.max_range:
                continue
            
            # Calculate angle
            angle = np.arctan2(pos[1], pos[0])
            
            # Check if within FoV
            if abs(angle) > self.angular_fov / 2:
                continue
            
            # Calculate radial velocity (projection onto line of sight)
            # Relative velocity = target velocity - ego velocity
            rel_velocity = vel - np.array([ego_velocity, 0])
            unit_vector = pos / range_m
            radial_velocity = np.dot(rel_velocity, unit_vector)
            
            detections.append((range_m, radial_velocity, angle, rcs))
        
        return detections
    
    def generate_range_doppler_map(self, detections, num_range_bins=256, num_doppler_bins=128):
        """
        Generate a Range-Doppler map from detections.
        
        Args:
            detections: List of (range, radial_velocity, angle, rcs) tuples
            num_range_bins: Number of range bins
            num_doppler_bins: Number of Doppler bins
        
        Returns:
            rd_map: Range-Doppler map (2D array)
            range_axis: Range values for each bin
            velocity_axis: Velocity values for each bin
        """
        # Create empty map
        rd_map = np.zeros((num_range_bins, num_doppler_bins))
        
        # Range and velocity axes
        range_axis = np.linspace(0, self.max_range, num_range_bins)
        max_velocity = 50  # m/s
        velocity_axis = np.linspace(-max_velocity, max_velocity, num_doppler_bins)
        
        # Populate map with detections
        for range_m, radial_velocity, angle, rcs in detections:
            # Find closest bins
            range_bin = np.argmin(np.abs(range_axis - range_m))
            velocity_bin = np.argmin(np.abs(velocity_axis - radial_velocity))
            
            # Add detection with RCS-based intensity
            rd_map[range_bin, velocity_bin] += rcs
        
        return rd_map, range_axis, velocity_axis


# Create 77 GHz FMCW Radar
radar = FMCWRadar(
    frequency=77e9,
    bandwidth=4e9,
    chirp_duration=100e-6,
    max_range=150,
    angular_fov=30,
    angular_resolution=2
)

print("=" * 70)
print("Radar Specifications:")
print("=" * 70)
print(f"Frequency: {radar.f0/1e9:.1f} GHz")
print(f"Bandwidth: {radar.B/1e9:.1f} GHz")
print(f"Wavelength: {radar.wavelength*1000:.2f} mm")
print(f"Range resolution: {radar.range_resolution:.3f} m")
print(f"Velocity resolution: {radar.velocity_resolution:.3f} m/s")
print(f"Max range: {radar.max_range} m")
print(f"Field of View: ±{np.rad2deg(radar.angular_fov/2):.1f}°")
print("=" * 70)

# Define scenario: Highway driving
ego_velocity = 25  # m/s (90 km/h)

# Objects: (position [x, y], velocity [vx, vy], RCS)
objects = [
    (np.array([50, 0]), np.array([20, 0]), 10),      # Vehicle ahead, slower (72 km/h)
    (np.array([80, -3.5]), np.array([30, 0]), 10),   # Vehicle in left lane, same speed
    (np.array([100, 3.5]), np.array([15, 0]), 10),   # Vehicle in right lane, slower
    (np.array([30, 2]), np.array([25, 0]), 5),       # Motorcycle nearby
    (np.array([120, 0]), np.array([25, 0]), 15),     # Truck ahead, same speed
]

# Detect targets
detections = radar.detect_targets(objects, ego_velocity=ego_velocity)

print(f"\n=== Detected Targets: {len(detections)} ===")
for i, (r, v_r, theta, rcs) in enumerate(detections):
    doppler_shift = radar.calculate_doppler_shift(v_r)
    print(f"Target {i+1}: Range={r:.1f}m, Radial Vel={v_r:.1f}m/s ({v_r*3.6:.1f}km/h), "
          f"Angle={np.rad2deg(theta):.1f}°, Doppler={doppler_shift/1000:.2f}kHz, RCS={rcs}m²")

# Generate Range-Doppler map
rd_map, range_axis, velocity_axis = radar.generate_range_doppler_map(detections)

# Visualization
fig = plt.figure(figsize=(16, 10))

# 1. Range-Doppler Map
ax1 = fig.add_subplot(2, 2, 1)
im1 = ax1.imshow(rd_map, aspect='auto', origin='lower', cmap='hot', 
                 extent=[velocity_axis[0], velocity_axis[-1], range_axis[0], range_axis[-1]])
ax1.set_xlabel('Radial Velocity (m/s)', fontsize=11, fontweight='bold')
ax1.set_ylabel('Range (m)', fontsize=11, fontweight='bold')
ax1.set_title('Range-Doppler Map (FMCW Radar)', fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3, color='white')
ax1.axvline(x=0, color='cyan', linestyle='--', linewidth=1.5, label='Zero velocity')
plt.colorbar(im1, ax=ax1, label='Intensity (RCS)')
ax1.legend()

# 2. Top-down view with radar FoV
ax2 = fig.add_subplot(2, 2, 2)

# Draw radar FoV
fov_angle = radar.angular_fov / 2
fov_arc = Wedge((0, 0), radar.max_range, np.rad2deg(-fov_angle), np.rad2deg(fov_angle), 
                facecolor='blue', alpha=0.1, edgecolor='blue', linewidth=2)
ax2.add_patch(fov_arc)

# Draw radar position
ax2.plot(0, 0, 'bs', markersize=15, label='Ego (Radar)', markeredgecolor='black', linewidth=2)

# Draw detected objects
for i, (r, v_r, theta, rcs) in enumerate(detections):
    x = r * np.cos(theta)
    y = r * np.sin(theta)
    
    # Color based on velocity (approaching=red, receding=green, stationary=yellow)
    if v_r > 1:
        color = 'red'
        marker = '^'
    elif v_r < -1:
        color = 'green'
        marker = 'v'
    else:
        color = 'yellow'
        marker = 'o'
    
    ax2.plot(x, y, marker=marker, markersize=10+rcs/2, color=color, 
            markeredgecolor='black', linewidth=1.5)
    ax2.text(x+5, y+3, f"{v_r:.1f}m/s", fontsize=9, color='black')

ax2.set_xlabel('X (m) - Forward', fontsize=11, fontweight='bold')
ax2.set_ylabel('Y (m) - Lateral', fontsize=11, fontweight='bold')
ax2.set_title('Radar Detection - Top View', fontsize=13, fontweight='bold')
ax2.set_aspect('equal')
ax2.grid(True, alpha=0.3)
ax2.set_xlim([-20, radar.max_range + 20])
ax2.set_ylim([-50, 50])
ax2.legend()

# Add custom legend for velocity
from matplotlib.lines import Line2D
legend_elements = [
    Line2D([0], [0], marker='^', color='w', markerfacecolor='red', markersize=10, 
           label='Approaching', markeredgecolor='black'),
    Line2D([0], [0], marker='v', color='w', markerfacecolor='green', markersize=10, 
           label='Receding', markeredgecolor='black'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor='yellow', markersize=10, 
           label='Stationary', markeredgecolor='black')
]
ax2.legend(handles=legend_elements, loc='upper right')

# 3. Range vs Radial Velocity scatter
ax3 = fig.add_subplot(2, 2, 3)
for i, (r, v_r, theta, rcs) in enumerate(detections):
    ax3.scatter(r, v_r, s=rcs*20, alpha=0.7, edgecolors='black', linewidth=1.5)
    ax3.text(r+2, v_r+0.5, f"T{i+1}", fontsize=9)

ax3.set_xlabel('Range (m)', fontsize=11, fontweight='bold')
ax3.set_ylabel('Radial Velocity (m/s)', fontsize=11, fontweight='bold')
ax3.set_title('Range vs Radial Velocity', fontsize=13, fontweight='bold')
ax3.axhline(y=0, color='gray', linestyle='--', linewidth=1, alpha=0.7)
ax3.grid(True, alpha=0.3)
ax3.set_xlim([0, radar.max_range])
ax3.set_ylim([-30, 30])

# 4. Doppler Shift Demonstration
ax4 = fig.add_subplot(2, 2, 4)

# Generate Doppler shift curve
velocities = np.linspace(-50, 50, 100)
doppler_shifts = radar.calculate_doppler_shift(velocities)

ax4.plot(velocities, doppler_shifts/1000, 'b-', linewidth=2, label='Doppler Shift')
ax4.axhline(y=0, color='gray', linestyle='--', linewidth=1)
ax4.axvline(x=0, color='gray', linestyle='--', linewidth=1)

# Mark detected targets
for i, (r, v_r, theta, rcs) in enumerate(detections):
    doppler = radar.calculate_doppler_shift(v_r)
    ax4.plot(v_r, doppler/1000, 'ro', markersize=10, markeredgecolor='black', linewidth=1.5)

ax4.set_xlabel('Radial Velocity (m/s)', fontsize=11, fontweight='bold')
ax4.set_ylabel('Doppler Shift (kHz)', fontsize=11, fontweight='bold')
ax4.set_title('Doppler Effect (77 GHz Radar)', fontsize=13, fontweight='bold')
ax4.grid(True, alpha=0.3)
ax4.legend()
ax4.set_xlim([-50, 50])

plt.tight_layout()
plt.show()

print("\n" + "=" * 70)
print("Doppler Shift Examples:")
print("=" * 70)
test_velocities = [30, 10, 0, -10, -30]  # m/s
for v in test_velocities:
    doppler = radar.calculate_doppler_shift(v)
    print(f"Velocity: {v:+.1f} m/s ({v*3.6:+.1f} km/h) → Doppler shift: {doppler/1000:+.2f} kHz")
print("=" * 70)

## 4. Sensor Fusion & Calibration

**Sensor fusion** combines data from multiple sensors to create a more accurate and robust perception of the environment than any single sensor could provide.

### Why Sensor Fusion?

Each sensor has complementary strengths and weaknesses:

| Sensor | Strengths | Weaknesses |
|--------|-----------|------------|
| **Camera** | High resolution, color, semantic info | No depth, weather-sensitive |
| **LiDAR** | Accurate 3D, works in dark | Expensive, no color, weather-sensitive |
| **Radar** | Direct velocity, all-weather, long range | Low angular resolution, no height |

**Synergy**: Camera provides semantic understanding, LiDAR provides accurate localization, Radar provides velocity and all-weather robustness.

---

### Sensor Fusion Architectures

#### **1. Early Fusion (Low-Level)**
- Combine raw sensor data before processing
- Example: Fuse LiDAR point cloud with camera pixels
- **Advantage**: Maximum information retention
- **Disadvantage**: Computationally expensive, requires precise calibration

#### **2. Late Fusion (High-Level)**
- Each sensor processes independently, combine detections
- Example: Merge object lists from camera, LiDAR, and radar
- **Advantage**: Modular, fault-tolerant
- **Disadvantage**: Information loss from independent processing

#### **3. Mid-Level Fusion (Feature-Level)**
- Fuse intermediate representations (features)
- Example: Combine camera features with LiDAR features
- **Advantage**: Balance between information and efficiency
- **Disadvantage**: Requires careful feature design

**Most common in production**: Late fusion with track-level association

---

### Sensor Calibration

**Calibration** determines the transformation between sensor frames to enable fusion.

#### **Intrinsic Calibration**
- Determines sensor's internal parameters
- **Camera**: Focal length, principal point, distortion coefficients
- **LiDAR**: Beam angles, range offsets
- **Radar**: Antenna phase offsets

#### **Extrinsic Calibration**
- Determines sensor position and orientation relative to vehicle frame
- Represented as homogeneous transformation matrix (4×4):

$$\mathbf{T}_{sensor}^{vehicle} = \begin{bmatrix} \mathbf{R} & \mathbf{t} \\ \mathbf{0}^T & 1 \end{bmatrix}$$

Where:
- **R**: 3×3 rotation matrix
- **t**: 3×1 translation vector (x, y, z position)

**Calibration methods**:
- **Manual**: Physical measurements
- **Target-based**: Use calibration targets (checkerboard, retroreflector)
- **Automatic**: Online calibration using scene structure

---

### Coordinate Frame Transformations

**Standard vehicle frame** (ISO 8855):
- **X**: Forward (longitudinal)
- **Y**: Left (lateral)
- **Z**: Up (vertical)
- **Origin**: Typically rear axle center

**Transformation pipeline**:
```
Sensor Frame → Vehicle Frame → World Frame
```

#### **Example: LiDAR to Camera Projection**

1. **Transform** LiDAR point from LiDAR frame to camera frame:
   $$\mathbf{p}_{cam} = \mathbf{T}_{lidar}^{cam} \mathbf{p}_{lidar}$$

2. **Project** 3D point to 2D image:
   $$\mathbf{p}_{image} = \mathbf{K} \cdot \mathbf{p}_{cam}$$

3. **Normalize** by depth and get pixel coordinates

---

### Time Synchronization

Sensors operate at different frequencies:
- **Camera**: 30-60 Hz
- **LiDAR**: 10-20 Hz  
- **Radar**: 10-50 Hz
- **IMU**: 100-1000 Hz
- **GNSS**: 1-10 Hz

**Synchronization strategies**:

#### **1. Hardware Synchronization**
- Use PTP (Precision Time Protocol) or GPS time
- All sensors trigger from common clock
- **Accuracy**: Sub-millisecond
- **Challenge**: Requires hardware support

#### **2. Software Synchronization**
- Timestamp each measurement
- Interpolate/extrapolate to common time
- **Advantage**: Works with any sensors
- **Disadvantage**: Introduces errors for fast-moving objects

#### **3. Buffering & Alignment**
- Buffer measurements within time window (e.g., 50ms)
- Align closest timestamps for fusion

**Critical for moving vehicles**: At 30 m/s, a 50ms delay = 1.5m position error!

---

### Sensor Fusion Algorithms

#### **1. Kalman Filter**
- Optimal for linear systems with Gaussian noise
- Used for sensor fusion and state estimation
- **Variants**: Extended Kalman Filter (EKF), Unscented Kalman Filter (UKF)

**State update**:
$$\hat{x}_{k} = \hat{x}_{k-1} + K_k (z_k - H \hat{x}_{k-1})$$

Where:
- **K_k**: Kalman gain (weighs prediction vs measurement)
- **z_k**: Measurement from sensor
- **H**: Measurement model

#### **2. Bayesian Occupancy Grid**
- Discretize space into grid cells
- Each cell has occupancy probability
- Fuse measurements from multiple sensors probabilistically

#### **3. Deep Learning Fusion**
- Neural networks learn optimal fusion strategy
- **Architectures**: BEVFusion, TransFusion, PointPainting
- **Input**: Multi-modal data (images, point clouds, radar)
- **Output**: Unified object detections or occupancy

---

### Data Association

**Problem**: Match detections from different sensors to same real-world object.

#### **Gating**
- Only consider associations within distance threshold
- **Mahalanobis distance**: Accounts for uncertainty

$$d_M = \sqrt{(\mathbf{z}_1 - \mathbf{z}_2)^T \mathbf{S}^{-1} (\mathbf{z}_1 - \mathbf{z}_2)}$$

#### **Association Algorithms**
- **Nearest Neighbor**: Match closest detections
- **Global Nearest Neighbor (GNN)**: Solve assignment problem
- **Joint Probabilistic Data Association (JPDA)**: Probabilistic matching
- **Multi-Hypothesis Tracking (MHT)**: Maintain multiple association hypotheses

---

### Sensor Fusion Pipeline

```
┌─────────┐  ┌─────────┐  ┌─────────┐
│ Camera  │  │ LiDAR   │  │ Radar   │
└────┬────┘  └────┬────┘  └────┬────┘
     │            │            │
     ▼            ▼            ▼
┌─────────────────────────────────┐
│   Calibration & Synchronization │
└────────────┬────────────────────┘
             ▼
┌──────────────────────────────────┐
│    Feature Extraction            │
│  (Objects, Lane Lines, etc.)     │
└────────────┬─────────────────────┘
             ▼
┌──────────────────────────────────┐
│     Data Association             │
│  (Match detections across sensors)│
└────────────┬─────────────────────┘
             ▼
┌──────────────────────────────────┐
│    State Estimation              │
│  (Kalman Filter, Tracking)       │
└────────────┬─────────────────────┘
             ▼
┌──────────────────────────────────┐
│   Fused Perception Output        │
│ (Object list: position, velocity,│
│  classification, uncertainty)    │
└──────────────────────────────────┘
```

---

### Advantages of Sensor Fusion

✅ **Robustness**: Redundancy handles sensor failures  
✅ **Accuracy**: Complementary info reduces uncertainty  
✅ **Completeness**: Better coverage of environment  
✅ **Reduced false positives**: Cross-validation between sensors  
✅ **All-weather operation**: Radar compensates when camera/LiDAR fail  

---

### Challenges

❌ **Calibration drift**: Sensors shift over time (vibration, temperature)  
❌ **Synchronization**: Temporal misalignment causes errors  
❌ **Computational cost**: Processing multiple sensors in real-time  
❌ **Conflicting measurements**: Resolving disagreements between sensors  
❌ **Scalability**: Adding more sensors increases complexity  

---

### Industry Examples

| Company | Sensor Suite | Fusion Strategy |
|---------|--------------|-----------------|
| **Waymo** | 5 LiDARs, 29 cameras, radars | Early + mid-level fusion |
| **Tesla** | 8 cameras, 12 ultrasonic, radar* | Vision-first, camera fusion |
| **Cruise** | 5 LiDARs, 21 cameras, 5 radars | Multi-modal late fusion |
| **Mobileye** | Cameras, radar, (LiDAR optional) | Camera-centric with radar |

*Tesla removed radar from newer vehicles (2023+) for vision-only approach

---

### Best Practices

1. **Calibrate regularly**: Check calibration every maintenance cycle
2. **Validate fusion output**: Compare against ground truth data
3. **Handle graceful degradation**: System should work with sensor failures
4. **Monitor sensor health**: Detect blockage, malfunction, degradation
5. **Use uncertainty estimates**: Track confidence in fused measurements
6. **Test in diverse conditions**: Rain, fog, night, tunnels, etc.

In [ ]:
# Simple Sensor Fusion Demonstration
# Late fusion: Combine object detections from camera, LiDAR, and radar

class SimpleSensorFusion:
    """
    Demonstrates late fusion of multi-sensor object detections.
    """
    def __init__(self, association_threshold=3.0):
        """
        Args:
            association_threshold: Maximum distance (m) for associating detections
        """
        self.threshold = association_threshold
    
    def associate_detections(self, camera_dets, lidar_dets, radar_dets):
        """
        Associate detections from different sensors using nearest neighbor.
        
        Args:
            camera_dets: List of (x, y, confidence, type) from camera
            lidar_dets: List of (x, y, z, confidence) from LiDAR
            radar_dets: List of (x, y, velocity, confidence) from radar
        
        Returns:
            fused_objects: List of fused object detections
        """
        fused_objects = []
        used_lidar = set()
        used_radar = set()
        
        # For each camera detection, find closest LiDAR and radar matches
        for cam_det in camera_dets:
            cam_x, cam_y, cam_conf, obj_type = cam_det
            cam_pos = np.array([cam_x, cam_y])
            
            # Find closest LiDAR detection
            best_lidar_idx = None
            best_lidar_dist = float('inf')
            for i, lidar_det in enumerate(lidar_dets):
                if i in used_lidar:
                    continue
                lidar_pos = np.array([lidar_det[0], lidar_det[1]])
                dist = np.linalg.norm(cam_pos - lidar_pos)
                if dist < best_lidar_dist and dist < self.threshold:
                    best_lidar_dist = dist
                    best_lidar_idx = i
            
            # Find closest radar detection
            best_radar_idx = None
            best_radar_dist = float('inf')
            for i, radar_det in enumerate(radar_dets):
                if i in used_radar:
                    continue
                radar_pos = np.array([radar_det[0], radar_det[1]])
                dist = np.linalg.norm(cam_pos - radar_pos)
                if dist < best_radar_dist and dist < self.threshold:
                    best_radar_dist = dist
                    best_radar_idx = i
            
            # Fuse detections
            fused_obj = self.fuse_object(cam_det, 
                                         lidar_dets[best_lidar_idx] if best_lidar_idx is not None else None,
                                         radar_dets[best_radar_idx] if best_radar_idx is not None else None)
            fused_objects.append(fused_obj)
            
            if best_lidar_idx is not None:
                used_lidar.add(best_lidar_idx)
            if best_radar_idx is not None:
                used_radar.add(best_radar_idx)
        
        return fused_objects
    
    def fuse_object(self, cam_det, lidar_det, radar_det):
        """
        Fuse information from camera, LiDAR, and radar for a single object.
        
        Returns:
            dict with fused properties: position, velocity, type, confidence
        """
        # Initialize with camera detection
        cam_x, cam_y, cam_conf, obj_type = cam_det
        
        # Weighted average of positions (higher weight for more confident sensors)
        positions = [(cam_x, cam_y, cam_conf)]
        confidences = [cam_conf]
        
        if lidar_det is not None:
            lidar_x, lidar_y, lidar_z, lidar_conf = lidar_det
            positions.append((lidar_x, lidar_y, lidar_conf * 1.2))  # LiDAR more reliable for position
            confidences.append(lidar_conf)
        
        if radar_det is not None:
            radar_x, radar_y, radar_vel, radar_conf = radar_det
            positions.append((radar_x, radar_y, radar_conf))
            confidences.append(radar_conf)
        
        # Weighted average position
        total_weight = sum(p[2] for p in positions)
        fused_x = sum(p[0] * p[2] for p in positions) / total_weight
        fused_y = sum(p[1] * p[2] for p in positions) / total_weight
        
        # Velocity from radar (only sensor that directly measures it)
        velocity = radar_det[2] if radar_det is not None else 0.0
        
        # Overall confidence (average with boost for multi-sensor)
        avg_conf = np.mean(confidences)
        num_sensors = 1 + (lidar_det is not None) + (radar_det is not None)
        fused_conf = min(1.0, avg_conf * (1 + 0.2 * (num_sensors - 1)))
        
        return {
            'position': (fused_x, fused_y),
            'velocity': velocity,
            'type': obj_type,
            'confidence': fused_conf,
            'num_sensors': num_sensors,
            'has_lidar': lidar_det is not None,
            'has_radar': radar_det is not None
        }


# Create fusion system
fusion = SimpleSensorFusion(association_threshold=3.0)

# Simulate detections from 3 sensors for a highway scene
# Camera detections: (x, y, confidence, type)
camera_dets = [
    (50, 0, 0.9, 'vehicle'),
    (80, -3.5, 0.85, 'vehicle'),
    (30, 2, 0.7, 'motorcycle'),
    (100, 3.5, 0.8, 'vehicle'),
]

# LiDAR detections: (x, y, z, confidence)
lidar_dets = [
    (51, 0.2, 0.75, 0.95),  # Matches camera detection 1
    (81, -3.3, 0.8, 0.9),   # Matches camera detection 2
    (99, 3.7, 0.9, 0.85),   # Matches camera detection 4
]

# Radar detections: (x, y, velocity_radial, confidence)
radar_dets = [
    (50.5, -0.1, -5, 0.9),    # Matches camera detection 1 (approaching)
    (79, -3.6, 0, 0.85),      # Matches camera detection 2 (same speed)
    (29, 2.2, -0.5, 0.75),    # Matches camera detection 3
]

# Perform sensor fusion
fused_objects = fusion.associate_detections(camera_dets, lidar_dets, radar_dets)

# Visualization
fig = plt.figure(figsize=(16, 12))

# 1. Camera-only detections
ax1 = fig.add_subplot(2, 2, 1)
ax1.plot(0, 0, 'bs', markersize=15, label='Ego vehicle', markeredgecolor='black', linewidth=2)
for i, (x, y, conf, obj_type) in enumerate(camera_dets):
    color = 'red' if obj_type == 'vehicle' else 'orange'
    ax1.plot(x, y, 'o', color=color, markersize=15, alpha=0.6, markeredgecolor='black', linewidth=2)
    ax1.text(x+3, y+1, f"Cam{i+1}\n{obj_type}\n{conf:.2f}", fontsize=9)
ax1.set_xlabel('X (m) - Forward', fontsize=11, fontweight='bold')
ax1.set_ylabel('Y (m) - Lateral', fontsize=11, fontweight='bold')
ax1.set_title('Camera Detections Only', fontsize=13, fontweight='bold')
ax1.set_aspect('equal')
ax1.grid(True, alpha=0.3)
ax1.set_xlim([-10, 110])
ax1.set_ylim([-10, 10])
ax1.legend()

# 2. LiDAR-only detections
ax2 = fig.add_subplot(2, 2, 2)
ax2.plot(0, 0, 'bs', markersize=15, label='Ego vehicle', markeredgecolor='black', linewidth=2)
for i, (x, y, z, conf) in enumerate(lidar_dets):
    ax2.plot(x, y, '^', color='green', markersize=15, alpha=0.6, markeredgecolor='black', linewidth=2)
    ax2.text(x+3, y+1, f"L{i+1}\n{conf:.2f}", fontsize=9)
ax2.set_xlabel('X (m) - Forward', fontsize=11, fontweight='bold')
ax2.set_ylabel('Y (m) - Lateral', fontsize=11, fontweight='bold')
ax2.set_title('LiDAR Detections Only', fontsize=13, fontweight='bold')
ax2.set_aspect('equal')
ax2.grid(True, alpha=0.3)
ax2.set_xlim([-10, 110])
ax2.set_ylim([-10, 10])
ax2.legend()

# 3. Radar-only detections
ax3 = fig.add_subplot(2, 2, 3)
ax3.plot(0, 0, 'bs', markersize=15, label='Ego vehicle', markeredgecolor='black', linewidth=2)
for i, (x, y, vel, conf) in enumerate(radar_dets):
    color = 'red' if vel < -1 else ('yellow' if abs(vel) <= 1 else 'green')
    ax3.plot(x, y, 's', color=color, markersize=15, alpha=0.6, markeredgecolor='black', linewidth=2)
    ax3.text(x+3, y+1, f"R{i+1}\nv={vel:.1f}m/s\n{conf:.2f}", fontsize=8)
ax3.set_xlabel('X (m) - Forward', fontsize=11, fontweight='bold')
ax3.set_ylabel('Y (m) - Lateral', fontsize=11, fontweight='bold')
ax3.set_title('Radar Detections Only', fontsize=13, fontweight='bold')
ax3.set_aspect('equal')
ax3.grid(True, alpha=0.3)
ax3.set_xlim([-10, 110])
ax3.set_ylim([-10, 10])
ax3.legend()

# 4. Fused detections
ax4 = fig.add_subplot(2, 2, 4)
ax4.plot(0, 0, 'bs', markersize=15, label='Ego vehicle', markeredgecolor='black', linewidth=2)

for i, obj in enumerate(fused_objects):
    x, y = obj['position']
    
    # Size based on confidence
    size = 20 + 10 * obj['confidence']
    
    # Color based on number of sensors
    if obj['num_sensors'] == 3:
        color = 'purple'
        edge_color = 'gold'
        edge_width = 3
    elif obj['num_sensors'] == 2:
        color = 'blue'
        edge_color = 'black'
        edge_width = 2
    else:
        color = 'gray'
        edge_color = 'black'
        edge_width = 1
    
    ax4.plot(x, y, 'D', color=color, markersize=size, alpha=0.7, 
            markeredgecolor=edge_color, linewidth=edge_width)
    
    # Label with details
    sensor_str = f"{'C' if True else ''}{'L' if obj['has_lidar'] else ''}{'R' if obj['has_radar'] else ''}"
    ax4.text(x+3, y+1.5, f"F{i+1} ({sensor_str})\n{obj['type']}\nv={obj['velocity']:.1f}m/s\nconf={obj['confidence']:.2f}", 
            fontsize=8, bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))

ax4.set_xlabel('X (m) - Forward', fontsize=11, fontweight='bold')
ax4.set_ylabel('Y (m) - Lateral', fontsize=11, fontweight='bold')
ax4.set_title('Fused Multi-Sensor Detections', fontsize=13, fontweight='bold')
ax4.set_aspect('equal')
ax4.grid(True, alpha=0.3)
ax4.set_xlim([-10, 110])
ax4.set_ylim([-10, 10])

# Custom legend
from matplotlib.lines import Line2D
legend_elements = [
    Line2D([0], [0], marker='D', color='w', markerfacecolor='purple', markersize=12, 
           label='3 sensors', markeredgecolor='gold', linewidth=3),
    Line2D([0], [0], marker='D', color='w', markerfacecolor='blue', markersize=12, 
           label='2 sensors', markeredgecolor='black', linewidth=2),
    Line2D([0], [0], marker='D', color='w', markerfacecolor='gray', markersize=12, 
           label='1 sensor', markeredgecolor='black', linewidth=1)
]
ax4.legend(handles=legend_elements, loc='upper right')

plt.tight_layout()
plt.show()

# Print fusion statistics
print("=" * 80)
print("SENSOR FUSION RESULTS")
print("=" * 80)
print(f"Camera detections: {len(camera_dets)}")
print(f"LiDAR detections: {len(lidar_dets)}")
print(f"Radar detections: {len(radar_dets)}")
print(f"Fused objects: {len(fused_objects)}")
print("\n" + "=" * 80)
print("Fused Object Details:")
print("=" * 80)

for i, obj in enumerate(fused_objects):
    sensors_used = []
    if True:  # Camera always present in this example
        sensors_used.append("Camera")
    if obj['has_lidar']:
        sensors_used.append("LiDAR")
    if obj['has_radar']:
        sensors_used.append("Radar")
    
    print(f"\nObject {i+1}:")
    print(f"  Position: ({obj['position'][0]:.2f}, {obj['position'][1]:.2f}) m")
    print(f"  Velocity: {obj['velocity']:.2f} m/s")
    print(f"  Type: {obj['type']}")
    print(f"  Confidence: {obj['confidence']:.2f}")
    print(f"  Sensors: {', '.join(sensors_used)} ({obj['num_sensors']} total)")
    print(f"  Confidence boost from fusion: +{(obj['confidence'] - 0.8)*100:.1f}%")

print("\n" + "=" * 80)
print("Key Insights:")
print("=" * 80)
print("✓ Multi-sensor detections have higher confidence")
print("✓ LiDAR provides accurate positioning")
print("✓ Radar provides direct velocity measurement")
print("✓ Camera provides object classification")
print("✓ Fusion reduces uncertainty and false positives")
print("=" * 80)

---

## Exercises

### Exercise 1: Camera Field of View

Given a camera with:
- Focal length fx = 1000 pixels
- Image width = 1920 pixels
- Calculate the horizontal field of view (FoV) in degrees.

**Hint**: Use the relationship: `FoV = 2 * arctan(width / (2 * fx))`

---

### Exercise 2: LiDAR Point Cloud Analysis

Using the LiDAR sensor defined earlier:
1. Modify the environment to add 2 more objects (your choice of position and type)
2. Run the LiDAR scan
3. Report:
   - Total number of points
   - Number of clusters detected
   - Distance to each cluster

---

### Exercise 3: Doppler Radar Calculation

A 77 GHz radar detects a vehicle:
- Range: 80 meters
- Doppler shift: +10.3 kHz (positive = approaching)

Calculate:
1. The radial velocity of the target (m/s)
2. The target's speed in km/h
3. If the ego vehicle is traveling at 100 km/h, what is the target vehicle's actual speed?

**Formula**: v_r = (f_d * c) / (2 * f_0)

---

### Exercise 4: Sensor Fusion Confidence

Given three sensor detections of the same object:
- Camera: position = (50, 2), confidence = 0.85
- LiDAR: position = (51, 1.8), confidence = 0.95
- Radar: position = (49.5, 2.1), confidence = 0.90

1. Calculate the fused position using weighted average (weight = confidence)
2. Calculate the overall confidence (use the formula from the fusion example)
3. Compare the result to using only the highest-confidence sensor

---

### Exercise 5: Sensor Comparison Table

Fill in the comparison table for a specific scenario: **Detecting a pedestrian at night in light rain at 30 meters distance**

| Sensor | Can Detect? | Accuracy | Confidence | Justification |
|--------|-------------|----------|------------|---------------|
| Camera | ? | ? | ? | ? |
| LiDAR | ? | ? | ? | ? |
| Radar | ? | ? | ? | ? |

Rate each as: High / Medium / Low / No

# Exercise Solutions

print("=" * 80)
print("EXERCISE SOLUTIONS - WEEK 4: SENSOR TECHNOLOGIES & FUSION")
print("=" * 80)

# Exercise 1: Camera Field of View
print("\n### Exercise 1: Camera Field of View ###\n")

fx = 1000  # pixels
width = 1920  # pixels

# Calculate horizontal FoV
fov_rad = 2 * np.arctan(width / (2 * fx))
fov_deg = np.rad2deg(fov_rad)

print(f"Given:")
print(f"  Focal length fx = {fx} pixels")
print(f"  Image width = {width} pixels")
print(f"\nCalculation:")
print(f"  FoV = 2 × arctan(width / (2 × fx))")
print(f"  FoV = 2 × arctan({width} / {2*fx})")
print(f"  FoV = 2 × arctan({width/(2*fx):.4f})")
print(f"  FoV = {fov_deg:.2f}°")
print(f"\nAnswer: The horizontal field of view is {fov_deg:.2f}°")

# Exercise 2: LiDAR Point Cloud Analysis
print("\n" + "=" * 80)
print("### Exercise 2: LiDAR Point Cloud Analysis ###\n")

# Create LiDAR sensor
lidar_ex2 = LiDARSensor(
    horizontal_resolution=0.4,
    vertical_resolution=2.0,
    horizontal_fov=360,
    max_range=50
)

# Modified environment with 2 additional objects
objects_ex2 = [
    (np.array([15, 0, 0.75]), 2.0, 'box'),      # Original: Vehicle ahead
    (np.array([20, -3, 0.75]), 2.0, 'box'),     # Original: Vehicle left lane
    (np.array([8, 2, 0.9]), 0.3, 'sphere'),     # Original: Pedestrian right
    (np.array([25, 5, 0.5]), 0.3, 'sphere'),    # Original: Pedestrian far right
    (np.array([35, 0, 0.75]), 2.5, 'box'),      # NEW: Large truck ahead
    (np.array([12, -4, 0.9]), 0.3, 'sphere'),   # NEW: Pedestrian left
]

print("Environment objects:")
for i, (center, size, shape) in enumerate(objects_ex2):
    print(f"  Object {i+1}: {shape} at ({center[0]:.1f}, {center[1]:.1f}, {center[2]:.1f})m, size={size}")

# Perform scan
point_cloud_ex2 = lidar_ex2.scan_environment(objects_ex2)
print(f"\nTotal points in scan: {len(point_cloud_ex2)}")

# Remove ground and cluster
non_ground_ex2 = lidar_ex2.remove_ground_plane(point_cloud_ex2, ground_threshold=-0.1)
print(f"Non-ground points: {len(non_ground_ex2)}")

if len(non_ground_ex2) > 0:
    cluster_labels_ex2 = lidar_ex2.cluster_points(non_ground_ex2, distance_threshold=1.5)
    num_clusters_ex2 = len(set(cluster_labels_ex2)) - (1 if -1 in cluster_labels_ex2 else 0)
    print(f"Number of clusters detected: {num_clusters_ex2}")
    
    print("\nCluster details:")
    for cluster_id in set(cluster_labels_ex2):
        if cluster_id == -1:
            continue
        cluster_points = non_ground_ex2[cluster_labels_ex2 == cluster_id]
        center = cluster_points[:, :3].mean(axis=0)
        distance = np.linalg.norm(center[:2])
        print(f"  Cluster {cluster_id}: Distance={distance:.2f}m, Points={len(cluster_points)}, "
              f"Center=({center[0]:.1f}, {center[1]:.1f}, {center[2]:.1f})m")

# Exercise 3: Doppler Radar Calculation
print("\n" + "=" * 80)
print("### Exercise 3: Doppler Radar Calculation ###\n")

f_0 = 77e9  # Hz (77 GHz)
c = 3e8  # m/s (speed of light)
f_d = 10.3e3  # Hz (10.3 kHz Doppler shift)
ego_speed_kmh = 100  # km/h

print(f"Given:")
print(f"  Radar frequency f_0 = 77 GHz")
print(f"  Doppler shift f_d = +10.3 kHz (approaching)")
print(f"  Ego vehicle speed = 100 km/h")

# 1. Calculate radial velocity
v_r = (f_d * c) / (2 * f_0)
print(f"\n1. Radial velocity:")
print(f"   v_r = (f_d × c) / (2 × f_0)")
print(f"   v_r = ({f_d} × {c}) / (2 × {f_0})")
print(f"   v_r = {v_r:.2f} m/s")

# 2. Convert to km/h
v_r_kmh = v_r * 3.6
print(f"\n2. Radial velocity in km/h:")
print(f"   v_r = {v_r:.2f} m/s × 3.6 = {v_r_kmh:.2f} km/h")

# 3. Calculate target's actual speed
ego_speed_mps = ego_speed_kmh / 3.6
target_speed_mps = ego_speed_mps - v_r  # Approaching means closing speed
target_speed_kmh = target_speed_mps * 3.6

print(f"\n3. Target vehicle's actual speed:")
print(f"   Ego speed = {ego_speed_kmh} km/h = {ego_speed_mps:.2f} m/s")
print(f"   Since target is approaching (positive Doppler):")
print(f"   Target speed = Ego speed - Radial velocity")
print(f"   Target speed = {ego_speed_mps:.2f} - {v_r:.2f} = {target_speed_mps:.2f} m/s")
print(f"   Target speed = {target_speed_kmh:.2f} km/h")
print(f"\n   Answer: Target is traveling at {target_speed_kmh:.1f} km/h in the same direction")

# Exercise 4: Sensor Fusion Confidence
print("\n" + "=" * 80)
print("### Exercise 4: Sensor Fusion Confidence ###\n")

# Sensor detections
camera_pos = np.array([50, 2])
camera_conf = 0.85
lidar_pos = np.array([51, 1.8])
lidar_conf = 0.95
radar_pos = np.array([49.5, 2.1])
radar_conf = 0.90

print("Given detections:")
print(f"  Camera: position = ({camera_pos[0]}, {camera_pos[1]}), confidence = {camera_conf}")
print(f"  LiDAR:  position = ({lidar_pos[0]}, {lidar_pos[1]}), confidence = {lidar_conf}")
print(f"  Radar:  position = ({radar_pos[0]}, {radar_pos[1]}), confidence = {radar_conf}")

# 1. Calculate fused position using weighted average
total_weight = camera_conf + lidar_conf + radar_conf
fused_x = (camera_pos[0] * camera_conf + lidar_pos[0] * lidar_conf + radar_pos[0] * radar_conf) / total_weight
fused_y = (camera_pos[1] * camera_conf + lidar_pos[1] * lidar_conf + radar_pos[1] * radar_conf) / total_weight
fused_pos = np.array([fused_x, fused_y])

print(f"\n1. Fused position (weighted average):")
print(f"   Total weight = {camera_conf} + {lidar_conf} + {radar_conf} = {total_weight}")
print(f"   x_fused = ({camera_pos[0]}×{camera_conf} + {lidar_pos[0]}×{lidar_conf} + {radar_pos[0]}×{radar_conf}) / {total_weight}")
print(f"   x_fused = {fused_x:.3f} m")
print(f"   y_fused = ({camera_pos[1]}×{camera_conf} + {lidar_pos[1]}×{lidar_conf} + {radar_pos[1]}×{radar_conf}) / {total_weight}")
print(f"   y_fused = {fused_y:.3f} m")
print(f"   Fused position = ({fused_x:.3f}, {fused_y:.3f}) m")

# 2. Calculate overall confidence
avg_conf = (camera_conf + lidar_conf + radar_conf) / 3
num_sensors = 3
fusion_boost = 1 + 0.2 * (num_sensors - 1)
fused_conf = min(1.0, avg_conf * fusion_boost)

print(f"\n2. Overall confidence:")
print(f"   Average confidence = ({camera_conf} + {lidar_conf} + {radar_conf}) / 3 = {avg_conf:.3f}")
print(f"   Fusion boost factor = 1 + 0.2 × (3 - 1) = {fusion_boost:.2f}")
print(f"   Fused confidence = {avg_conf:.3f} × {fusion_boost:.2f} = {fused_conf:.3f}")

# 3. Compare to highest-confidence sensor only
best_sensor = "LiDAR"
best_conf = lidar_conf
best_pos = lidar_pos

print(f"\n3. Comparison:")
print(f"   Best single sensor: {best_sensor}")
print(f"   - Position: ({best_pos[0]}, {best_pos[1]})")
print(f"   - Confidence: {best_conf:.3f}")
print(f"   \n   Fused result:")
print(f"   - Position: ({fused_x:.3f}, {fused_y:.3f})")
print(f"   - Confidence: {fused_conf:.3f}")
print(f"   \n   Improvement:")
print(f"   - Confidence gain: +{(fused_conf - best_conf)*100:.1f}%")
print(f"   - Position error vs LiDAR: {np.linalg.norm(fused_pos - lidar_pos):.3f} m")
print(f"   \n   Conclusion: Fusion provides higher confidence and averages out position errors!")

# Exercise 5: Sensor Comparison Table
print("\n" + "=" * 80)
print("### Exercise 5: Sensor Comparison - Pedestrian at Night in Light Rain (30m) ###\n")

comparison_data = {
    'Camera': {
        'Can Detect': 'Low',
        'Accuracy': 'Low',
        'Confidence': 'Low',
        'Justification': 'Poor lighting (night) and rain reduce visibility. May miss pedestrian or have low confidence.'
    },
    'LiDAR': {
        'Can Detect': 'Medium',
        'Accuracy': 'Medium',
        'Confidence': 'Medium',
        'Justification': 'Works in darkness, but light rain causes noise and reduces effective range. Can detect at 30m but with degraded accuracy.'
    },
    'Radar': {
        'Can Detect': 'Medium',
        'Accuracy': 'Low',
        'Confidence': 'Medium',
        'Justification': 'Unaffected by darkness or rain. However, pedestrians have small radar cross-section (RCS), making detection challenging. Low angular resolution.'
    }
}

print("Scenario: Detecting a pedestrian at night in light rain at 30 meters distance\n")
print(f"{'Sensor':<10} {'Can Detect?':<15} {'Accuracy':<12} {'Confidence':<12} {'Justification'}")
print("-" * 110)

for sensor, data in comparison_data.items():
    print(f"{sensor:<10} {data['Can Detect']:<15} {data['Accuracy']:<12} {data['Confidence']:<12} {data['Justification']}")

print("\n" + "=" * 80)
print("Key Takeaways:")
print("=" * 80)
print("✓ No single sensor is reliable in this challenging scenario")
print("✓ Sensor fusion is CRITICAL for robust detection")
print("✓ Camera + LiDAR + Radar together provide complementary information")
print("✓ This demonstrates why autonomous vehicles need multi-sensor suites")
print("=" * 80)

---

## References

### Textbooks

1. **Thrun, S., Burgard, W., & Fox, D.** (2005). *Probabilistic Robotics*. MIT Press.
   - Chapters 6-7: Sensor models and observation models
   - Gold standard for sensor modeling and Bayesian filtering

2. **Szeliski, R.** (2022). *Computer Vision: Algorithms and Applications* (2nd ed.). Springer.
   - Chapter 2: Image formation and camera models
   - Chapter 11: Structure from motion
   - Available online: https://szeliski.org/Book/

3. **Levinson, J., & Thrun, S.** (2014). *Robust Vehicle Localization in Urban Environments Using Probabilistic Maps*.
   - Sensor fusion techniques for autonomous driving

### Research Papers

#### Camera

4. **DeTone, D., Malisiewicz, T., & Rabinovich, A.** (2018). "SuperPoint: Self-Supervised Interest Point Detection and Description." *CVPR*.
   - Modern feature detection for camera-based perception

5. **Philion, J., & Fidler, S.** (2020). "Lift, Splat, Shoot: Encoding Images from Arbitrary Camera Rigs by Implicitly Unprojecting to 3D." *ECCV*.
   - Camera-to-BEV transformation for autonomous driving

#### LiDAR

6. **Qi, C. R., Yi, L., Su, H., & Guibas, L. J.** (2017). "PointNet++: Deep Hierarchical Feature Learning on Point Sets in a Metric Space." *NeurIPS*.
   - Deep learning on LiDAR point clouds

7. **Geiger, A., Lenz, P., & Urtasun, R.** (2012). "Are We Ready for Autonomous Driving? The KITTI Vision Benchmark Suite." *CVPR*.
   - Standard benchmark for LiDAR and camera evaluation

#### Radar

8. **Major, B., et al.** (2019). "Vehicle Detection With Automotive Radar Using Deep Learning on Range-Azimuth-Doppler Tensors." *ICCV Workshops*.
   - Deep learning for automotive radar

9. **Bilik, I., et al.** (2019). "Automotive MIMO Radar for Urban Environments." *IEEE Radar Conference*.
   - Modern 4D imaging radar technology

#### Sensor Fusion

10. **Liang, M., et al.** (2019). "Multi-Task Multi-Sensor Fusion for 3D Object Detection." *CVPR*.
    - State-of-the-art multi-sensor fusion

11. **Liu, Z., et al.** (2023). "BEVFusion: Multi-Task Multi-Sensor Fusion with Unified Bird's-Eye View Representation." *ICRA*.
    - Unified BEV representation for camera + LiDAR

12. **Chen, X., et al.** (2017). "Multi-View 3D Object Detection Network for Autonomous Driving." *CVPR*.
    - MV3D: Early work on multi-sensor 3D detection

### Industry Reports

13. **Yole Développement** (2023). "Automotive LiDAR Market and Technology Report."
    - Market trends and technology comparison

14. **IEEE Spectrum** (2022). "Sensor Fusion: The Key to Autonomous Driving."
    - Industry overview of sensor suites

### Online Resources

15. **Udacity Self-Driving Car Nanodegree**: https://www.udacity.com/course/self-driving-car-engineer-nanodegree--nd0013
    - Sensor fusion and tracking projects

16. **KITTI Dataset**: http://www.cvlibs.net/datasets/kitti/
    - Benchmark dataset with camera, LiDAR, GPS/IMU data

17. **nuScenes Dataset**: https://www.nuscenes.org/
    - Full sensor suite dataset (6 cameras, 5 radars, 1 LiDAR)

18. **Waymo Open Dataset**: https://waymo.com/open/
    - Large-scale dataset from Waymo's autonomous vehicles

19. **OpenPCDet**: https://github.com/open-mmlab/OpenPCDet
    - Open-source toolbox for LiDAR-based 3D object detection

### Standards & Regulations

20. **ISO 26262**: Road vehicles - Functional safety
    - Safety requirements for automotive sensors

21. **SAE J3016**: Taxonomy and Definitions for Terms Related to Driving Automation Systems
    - Defines sensor requirements for each autonomy level

### Sensor Manufacturers (for specifications)

22. **Velodyne Lidar**: https://velodynelidar.com/
    - HDL-64E, VLS-128 specifications

23. **Luminar Technologies**: https://luminartech.com/
    - Iris LiDAR specifications

24. **Continental Automotive**: https://www.continental-automotive.com/
    - ARS540 radar specifications (77 GHz FMCW)

25. **Mobileye**: https://www.mobileye.com/
    - EyeQ camera-based perception systems

---

### Recommended Reading Order

**For beginners:**
1. Start with Thrun's *Probabilistic Robotics* Chapter 6
2. Explore KITTI dataset to see real sensor data
3. Read Geiger et al. (2012) for context

**For practitioners:**
1. Szeliski's *Computer Vision* for camera fundamentals
2. Liu et al. (2023) BEVFusion for modern fusion architectures
3. Explore nuScenes dataset for multi-sensor projects

**For researchers:**
1. Survey recent CVPR/ICCV papers on sensor fusion
2. Study Waymo Open Dataset papers
3. Follow arXiv for latest developments (cs.CV, cs.RO)

---

*Note: Some links and papers may require institutional access. Many papers are available on arXiv.org*